In [2]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import MobileNetV2, ResNet50
from tensorflow.keras.models import Model
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, Dropout
from tensorflow.keras.optimizers import Adam

In [ ]:
# data augmentation
datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    validation_split=0.2  
)

# training set
train_generator = datagen.flow_from_directory(
    "02-PetImages",                
    target_size=(224,224),
    batch_size=32,
    class_mode="binary",
    subset="training"      #
)

# validation set
val_generator = datagen.flow_from_directory(
    "02-PetImages",
    target_size=(224,224),
    batch_size=32,
    class_mode="binary",
    subset="validation"   
)

Found 20000 images belonging to 2 classes.
Found 4998 images belonging to 2 classes.


In [ ]:
# 2.load pre-trained model 
base_model = MobileNetV2(weights="imagenet", include_top=False, input_shape=(224,224,3))

# freeze the base model
base_model.trainable = False


In [ ]:
# 3. add custom classification head
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dropout(0.5)(x)  # add dropout for regularization
predictions = Dense(1, activation="sigmoid")(x)  # binary classification

model = Model(inputs=base_model.input, outputs=predictions)

In [ ]:
# 4. compile the model
model.compile(optimizer=Adam(learning_rate=0.0001),
              loss="binary_crossentropy",
              metrics=["accuracy"])

In [7]:
print(tf.config.list_physical_devices('GPU'))


[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [8]:
history = model.fit(
    train_generator,
    validation_data=val_generator,
    epochs=5,
    workers=8
)

Epoch 1/5
 50/625 [=>............................] - ETA: 31s - loss: 0.8334 - accuracy: 0.5131

c:\Users\User\anaconda3\envs\tf-gpu\lib\site-packages\PIL\TiffImagePlugin.py:950: UserWarning: Truncated File Read
  warnings.warn(str(msg))


625/625 [==============================] - 52s 76ms/step - loss: 0.3934 - accuracy: 0.8209 - val_loss: 0.1617 - val_accuracy: 0.9484
Epoch 2/5
625/625 [==============================] - 146s 232ms/step - loss: 0.1728 - accuracy: 0.9380 - val_loss: 0.1176 - val_accuracy: 0.9582
Epoch 3/5
625/625 [==============================] - 147s 234ms/step - loss: 0.1349 - accuracy: 0.9491 - val_loss: 0.0977 - val_accuracy: 0.9656
Epoch 4/5
625/625 [==============================] - 145s 230ms/step - loss: 0.1163 - accuracy: 0.9560 - val_loss: 0.0874 - val_accuracy: 0.9692
Epoch 5/5
625/625 [==============================] - 145s 230ms/step - loss: 0.1105 - accuracy: 0.9572 - val_loss: 0.0863 - val_accuracy: 0.9662


In [9]:
loss, acc = model.evaluate(val_generator)
print("验证集准确率:", acc)

157/157 [==============================] - 101s 644ms/step - loss: 0.0845 - accuracy: 0.9698
验证集准确率: 0.9697878956794739


In [ ]:
# 6. fine-tuning
base_model.trainable = True
for layer in base_model.layers[:-30]:  # unfreeze last 30 layers
    layer.trainable = False

model.compile(optimizer=Adam(learning_rate=0.00001),  # lower learning rate
              loss="binary_crossentropy",
              metrics=["accuracy"])

# continue training
history_finetune = model.fit(
    train_generator,
    validation_data=val_generator,
    epochs=5,
    workers=8
)

# 7. evaluate the model
loss, acc = model.evaluate(val_generator)
print("验证集准确率:", acc)

Epoch 1/5
625/625 [==============================] - 144s 216ms/step - loss: 0.1676 - accuracy: 0.9316 - val_loss: 0.0681 - val_accuracy: 0.9752
Epoch 2/5
625/625 [==============================] - 135s 214ms/step - loss: 0.1034 - accuracy: 0.9582 - val_loss: 0.0577 - val_accuracy: 0.9778
Epoch 3/5
625/625 [==============================] - 128s 202ms/step - loss: 0.0875 - accuracy: 0.9650 - val_loss: 0.0570 - val_accuracy: 0.9766
Epoch 4/5
625/625 [==============================] - 79s 125ms/step - loss: 0.0789 - accuracy: 0.9685 - val_loss: 0.0515 - val_accuracy: 0.9808
Epoch 5/5
157/157 [==============================] - 97s 615ms/step - loss: 0.0505 - accuracy: 0.9794
验证集准确率: 0.9793917536735535
